In [8]:
import nipype.pipeline.engine as pe
import nipype.interfaces.fsl as fsl
import nipype.interfaces.io as nio
import nipype.interfaces.ants as ants
import nipype.interfaces.utility as util
from nipype.interfaces.c3 import C3dAffineTool


In [9]:
import glob

In [10]:
templates = {'mean_epi':'/home/gdholla1/data/daphne/preprocessed/feat_preprocess/mean/_subject_id_{subject_id}/_fwhm_0.0/run1_dtype_mcf_mask_gms_mean.nii.gz',
              't1_weighted':'/home/gdholla1/data/daphne/clean/{subject_id}/t1.nii.gz',
             'epi2t1_weighted':'/home/gdholla1/data/daphne/registration/slab2t1_weighted/_subject_id_{subject_id}/run1_dtype_mcf_mask_gms_mean_flirt_whole_brain_epi_flirt.mat',
             't1_weighted2mni':'/home/gdholla1/data/daphne/registration/t1_weighted2mni/_subject_id_{subject_id}/transformComposite.h5',}

workflow = pe.Workflow(name='register2mni', base_dir='/home/gdholla1/workflow_folders/')

subject_ids = [fn.split('/')[-1] for fn in glob.glob('/home/gdholla1/data/daphne/clean/*')]

identity = pe.Node(util.IdentityInterface(fields=['subject_id']), name='identity')
identity.iterables = [('subject_id', subject_ids)]
             
selector = pe.Node(nio.SelectFiles(templates), name='selector')


# selector.iterables = [('subject_id', ['S02'])]
             
workflow.connect(identity, 'subject_id', selector, 'subject_id')


convert2itk = pe.Node(C3dAffineTool(),
                  name='convert2itk')
convert2itk.inputs.fsl2ras = True
convert2itk.inputs.itk_transform = True

workflow.connect(selector, 'epi2t1_weighted', convert2itk, 'transform_file')
workflow.connect(selector, 'mean_epi', convert2itk, 'source_file')
workflow.connect(selector, 't1_weighted', convert2itk, 'reference_file')

transform_merger = pe.Node(util.Merge(2), name='transform_merger')
workflow.connect(selector, 't1_weighted2mni', transform_merger, 'in1')
workflow.connect(convert2itk, 'itk_transform', transform_merger, 'in2')


mean_epi2mni = pe.Node(ants.ApplyTransforms(), name='reference_file')
mean_epi2mni.inputs.reference_image = '/usr/share/fsl/data/standard/MNI152_T1_2mm_brain.nii.gz'
workflow.connect(selector, 'mean_epi', mean_epi2mni, 'input_image')
workflow.connect(transform_merger, 'out', mean_epi2mni, 'transforms')
             
level2_templates = {'cope':'/home/gdholla1/data/daphne/fit_glm2/copes/_subject_id_{subject_id}/_fwhm_5.0/_shift_0.0/_modelestimate0/cope{contrast}.nii.gz',
                    'varcope':'/home/gdholla1/data/daphne/fit_glm2/varcopes/_subject_id_{subject_id}/_fwhm_5.0/_shift_0.0/_modelestimate0/varcope{contrast}.nii.gz',}

level2_selector = pe.Node(nio.SelectFiles(level2_templates), name='level2_selector')
level2_selector.iterables = [('contrast', range(1, 8))]

workflow.connect(identity, 'subject_id', level2_selector, 'subject_id')


cope_transformer = mean_epi2mni.clone('cope_transformer')
cope_transformer.inputs.reference_image = '/usr/share/fsl/data/standard/MNI152_T1_2mm_brain.nii.gz'
workflow.connect(level2_selector, 'cope', cope_transformer, 'input_image')
workflow.connect(transform_merger, 'out', cope_transformer, 'transforms')

varcope_transformer = mean_epi2mni.clone('varcope_transformer')
cope_transformer.inputs.reference_image = '/usr/share/fsl/data/standard/MNI152_T1_2mm_brain.nii.gz'
workflow.connect(level2_selector, 'varcope', varcope_transformer, 'input_image')
workflow.connect(transform_merger, 'out', varcope_transformer, 'transforms')

ds = pe.Node(nio.DataSink(), name='datasink')
ds.inputs.base_directory = '/home/gdholla1/data/daphne/level3_2'

workflow.connect(mean_epi2mni, 'output_image', ds, 'mean_epi2mni')
workflow.connect(cope_transformer, 'output_image', ds, 'cope2mni')
workflow.connect(varcope_transformer, 'output_image', ds, 'varcope2mni')


workflow.run()

In [11]:
import nibabel as nb

In [12]:
fns = glob.glob('/home/gdholla1/data/daphne/level3/mean_epi2mni/_subject_id_*/run1_dtype_mcf_mask_gms_mean_trans.nii.gz')
len(fns)

12

In [4]:
means = np.array([nb.load(fn).get_data() for fn in fns])

mean_of_means = means.mean(0)
nb.save(nb.Nifti1Image(mean_of_means, nb.load(fns[0]).get_affine()), '/home/gdholla1/data/daphne/level3/mean_of_means.nii.gz')

mask = ((means > 0).sum(0) == 12).astype(int)
nb.save(nb.Nifti1Image(mask, nb.load(fns[0]).get_affine()), '/home/gdholla1/data/daphne/level3/mask.nii.gz')


NameError: name 'fns' is not defined

# Level 3 analysis

In [13]:
def get_dofvolumes(dof_files, cope_files):
    import os
    import nibabel as nb
    import numpy as np
    print cope_files
    img = nb.load(cope_files)

    out_data = np.zeros(img.shape)
    out_data[:] = np.loadtxt(dof_files)

    filename = os.path.join(os.getcwd(), 'dof_file.nii.gz')
    newimg = nb.Nifti1Image(out_data, None, img.header)
    newimg.to_filename(filename)
    return filename

gendof = pe.MapNode(util.Function(input_names=['dof_files', 'cope_files'],
                               output_names=['dof_volume'],
                               function=get_dofvolumes),
                 iterfield=['dof_files', 'cope_files'],
                 name='gendofvolume')

In [14]:
templates = {'cope':'/home/gdholla1/data/daphne/level3_2/cope2mni/_subject_id_*/_contrast_{contrast}/cope{contrast}_trans.nii.gz',
             'varcope':'/home/gdholla1/data/daphne/level3_2/varcope2mni/_subject_id_*/_contrast_{contrast}/varcope{contrast}_trans.nii.gz',
             'dof':'/home/gdholla1/data/daphne/fit_glm2/dof_files/_subject_id_*/_fwhm_5.0/_shift_0.0/_modelestimate0/dof'}

workflow = pe.Workflow(name='level3_daphne', base_dir='/home/gdholla1/workflow_folders/')

            
selector = pe.Node(nio.SelectFiles(templates), name='selector')
selector.iterables = [('contrast', range(1, 8))]

from gilles_workflows import create_random_effects_workflow

In [15]:
workflow.connect(selector, 'cope', gendof, 'cope_files')
workflow.connect(selector, 'dof', gendof, 'dof_files')

randomfx = create_random_effects_workflow()

workflow.connect(selector, 'cope', randomfx, 'inputspec.cope_files')
workflow.connect(selector, 'varcope', randomfx, 'inputspec.varcope_files')
workflow.connect(gendof, 'dof_volume', randomfx, 'inputspec.tdof_files')

randomfx.inputs.inputspec.mask_file = '/home/gdholla1/data/daphne/level3/mask.nii.gz'

ds = pe.Node(nio.DataSink(), name='datasink')
ds.inputs.base_directory = '/home/gdholla1/data/daphne/level3_2'

for key in randomfx.outputs.outputspec.get().keys():
    workflow.connect(randomfx, 'outputspec.%s' % key, ds, key)


workflow.run()

/home/gdholla1/data/daphne/level3_2/cope2mni/_subject_id_S00/_contrast_2/cope2_trans.nii.gz
/home/gdholla1/data/daphne/level3_2/cope2mni/_subject_id_S01/_contrast_2/cope2_trans.nii.gz
/home/gdholla1/data/daphne/level3_2/cope2mni/_subject_id_S02/_contrast_2/cope2_trans.nii.gz
/home/gdholla1/data/daphne/level3_2/cope2mni/_subject_id_S03/_contrast_2/cope2_trans.nii.gz
/home/gdholla1/data/daphne/level3_2/cope2mni/_subject_id_S04/_contrast_2/cope2_trans.nii.gz
/home/gdholla1/data/daphne/level3_2/cope2mni/_subject_id_S06/_contrast_2/cope2_trans.nii.gz
/home/gdholla1/data/daphne/level3_2/cope2mni/_subject_id_S07/_contrast_2/cope2_trans.nii.gz
/home/gdholla1/data/daphne/level3_2/cope2mni/_subject_id_S08/_contrast_2/cope2_trans.nii.gz
/home/gdholla1/data/daphne/level3_2/cope2mni/_subject_id_S09/_contrast_2/cope2_trans.nii.gz
/home/gdholla1/data/daphne/level3_2/cope2mni/_subject_id_S11/_contrast_2/cope2_trans.nii.gz
/home/gdholla1/data/daphne/level3_2/cope2mni/_subject_id_S13/_contrast_2/cope2_t

In [72]:

    print key, value

txt_index_file None
zstats None
thresholded_z_stats None


In [60]:
randomfx.inputs.inputspec


cope_files = <undefined>
fdr_q = 0.05
mask_file = <undefined>
tdof_files = <undefined>
varcope_files = <undefined>

In [59]:
workflow.run()

In [ ]:
/home/gdholla1/data/daphne/level3/varcope2mni/_subject_id_S00/_contrast_3/varcope3_trans.nii.gz

In [4]:
transformer = ants.ApplyTransforms()

In [ ]:
from nipype.interfaces.c3 import C3dAffineTool

In [5]:
transformer.inputs.input_image = '/home/gdholla1/data/daphne/preprocessed/mean_signal/_subject_id_S00/_fwhm_0.0/_tsnr0/run1_dtype_mcf_mask_gms_tempfilt_maths_tsnr_mean.nii.gz'
transformer.inputs.reference_image = '/home/gdholla1/data/daphne/clean/S00/t1.nii.gz'
transformer.inputs.transforms = '/home/gdholla1/data/daphne/registration/slab2t1_weighted/_subject_id_S00/run1_dtype_mcf_mask_gms_mean_flirt_whole_brain_epi_flirt.mat'

r = transformer.run()

RuntimeError: Command:
antsApplyTransforms --default-value 0 --input /home/gdholla1/data/daphne/preprocessed/mean_signal/_subject_id_S00/_fwhm_0.0/_tsnr0/run1_dtype_mcf_mask_gms_tempfilt_maths_tsnr_mean.nii.gz --interpolation Linear --output run1_dtype_mcf_mask_gms_tempfilt_maths_tsnr_mean_trans.nii.gz --reference-image /home/gdholla1/data/daphne/clean/S00/t1.nii.gz --transform /home/gdholla1/data/daphne/registration/slab2t1_weighted/_subject_id_S00/run1_dtype_mcf_mask_gms_mean_flirt_whole_brain_epi_flirt.mat
Standard output:

Standard error:
Segmentation fault (core dumped)
Return code: 139
Interface ApplyTransforms failed to run. 

In [ ]:
r.outputs

In [ ]:
transformer.inputs.input_image = '/home/gdholla1/data/daphne/clean/S00/t1.nii.gz'
transformer.inputs.reference_image = '/usr/share/fsl/data/standard/MNI152_T1_2mm_brain.nii.gz'
transformer.inputs.transforms = 